In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint
from tensorflow.keras.applications.resnet_v2 import ResNet50V2
from tensorflow.keras.utils import Sequence
import pickle
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Flatten, Dropout, Input, GlobalAveragePooling2D, Flatten
import orjson
from threading import Thread, Lock
from queue import Queue

import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [2]:
with open('../dataset_test_files.pkl', 'rb') as fp:
    df_test = pickle.load(fp)
df_test.head()

,filename,kind,defect,size,class_name,class
3,D:\desenvolvimento\datasets\TRAIN1200 -1200_CO...,R,S,G,RS,3
4,D:\desenvolvimento\datasets\TRAIN1200 -1200_CO...,R,S,G,RS,3
11,D:\desenvolvimento\datasets\TRAIN1200 -1200_CO...,R,S,G,RS,3
15,D:\desenvolvimento\datasets\TRAIN1200 -1200_CO...,R,S,G,RS,3
20,D:\desenvolvimento\datasets\TRAIN1200 -1200_CO...,R,S,G,RS,3


In [3]:
gen_test = ImageDataGenerator().flow_from_dataframe(df_test, y_col='class_name', batch_size=64)

Found 360 validated image filenames belonging to 4 classes.


In [4]:
input_tensor = Input(shape=(256, 256, 3))
base_model = ResNet50V2(False, None, input_tensor=input_tensor)
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
# and a logistic layer -- let's say we have 4 classes
predictions = Dense(4, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)
model

In [5]:
model.load_weights('model_da_resnet50v2.084-0.7852.h5')

In [6]:
model.compile(optimizer='adam', # Rotina de otimização, que informa a ml como ajustar o valor dos parâmetros para minimizar o erro.
              loss='categorical_crossentropy', # Função de erro que diz o quão erradas estão nossas predições
              metrics=['accuracy']) # Lista de métricas para avaliar o nosso modelo

In [7]:
model.evaluate(gen_test)

6/6 [==============================] - 9s 2s/step - loss: 0.7860 - accuracy: 0.7833


[0.7860410114129385, 0.78333336]